# FAISS 실습

훈련은 미션 5에서 훈련해둔 인코드를 가져와서 수행

두 가지 Corpus 에 대해서 평가

평가하고자 하는 Corpus: korquad - 검증 데이터셋

평가하고자 하는 Corpus: wikipedia 문서들

### Requirements

In [1]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.4.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1
!pip install faiss-cpu > /dev/null 2>&1

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


In [2]:
import torch
import random
from pprint import pprint

## Passage retrieval 준비하기

In [10]:
# 1. 미리 학습해둔 encoder file 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx" -O encoders.tar.gz && rm -rf /tmp/cookies.txt
# 2. .tar.gz file 압축해제 
!tar -xf ./encoders.tar.gz

In [16]:
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

# TODO: load a model trained at Daily Mission 4
# use from_pretrained
# put your model to gpu by .cuda()
p_encoder = BertEncoder.from_pretrained('./p_encoder').cuda()
q_encoder = BertEncoder.from_pretrained('./q_encoder').cuda()

# You can use "bert-base-multilingual-cased" for tokenizer, if you did not save tokenizer at Daily Mission 4.
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Search corpus: KorQuAD validation context

In [17]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

Reusing dataset squad_kor_v1 (C:\Users\AI_15\.cache\huggingface\datasets\squad_kor_v1\squad_kor_v1\1.0.0\18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

9606

In [18]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

Passage encoder를 활용하여 passage dense embedding 생성

In [19]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm.auto import tqdm, trange

eval_batch_size = 8

# Construct dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

Iteration:   0%|          | 0/120 [00:00<?, ?it/s]

In [20]:
import numpy as np
p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)

(960, 768)

Question encoder를 활용해여 question dense embedding 생성

In [21]:
np.random.seed(1)

sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은?',
 '금강산의 겨울 이름은?',
 '유관순 열사는 당시 어떤 종교를 믿고 있었는가?',
 '1997년 10월 23일, 국회 본회의 대표 연설에서 전두환, 노태우 전 대통령에 대한 사면을 촉구한 새정치 국민회의 의원은?',
 '셰르징거가 찾아왔다가 우연히 푸시캣 돌스에 영입된 곳은?']

In [22]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

## GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [23]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [24]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[843, 241, 625,  ..., 885,  49, 127],
        [428, 340, 638,  ..., 557, 401, 719],
        [847, 339, 934,  ..., 130, 839, 406],
        [226, 216, 346,  ..., 279, 770, 348],
        [210, 917, 680,  ..., 130, 325, 760]], device='cuda:0')
--- 0.01795196533203125 seconds ---


In [25]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with score 57.2859
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정

## FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [28]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

In [29]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [30]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0009968280792236328 seconds ---


In [31]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[162.76018 166.36844 170.21155 172.91968 173.79454]
 [139.74776 142.22607 142.95209 151.57596 153.51622]
 [134.44823 137.05942 137.79669 138.13394 138.43002]
 [130.65106 130.69214 130.84744 134.1312  135.26862]
 [209.12695 209.20168 210.0505  212.58987 212.85425]]


=======[Index of Top-5 Passages]=======
[[882 742 507 619 104]
 [428 882 340 213 570]
 [ 12 728 865 720 591]
 [216 346 203 226  64]
 [480 279 629 657 641]]


In [32]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with distance 162.7602
이 지역은 대해 때문에 세상에 알려진 지 300년 밖에 되지 않았는데, 숙종 24년(1698년) 고성 군수로 있던 남택하(南宅夏)가 찾아내고 “금강산의 얼굴빛과 같다.” 하여 해금강이라 이름 붙였다. 본래 해안 암벽, 바위섬, 자연호, 모래사장, 하천이 어우러진 경승지다. 이중 개방된 곳은 삼일포와 향로봉이며, 관동 팔경의 하나인 총석정은 개방되어 있지 않다. 삼일포는 남한의 화진포와 송지호같이 석호(潟湖)이며, 총 넓이는 0.79km에 달한다. 이 호수에는 전설에 따르면 신선 또는 화랑들이 경치가 너무 좋아 3일 동안 머물고 갔기 때문에 삼일포라 한다. 봉래대에서 삼일포 전경을 볼 수 있다. 소가 누운 모양이라고 해서 와우섬이라 이름 붙은 큰 섬을 비롯해, 3개의 작은 섬이 떠있다. 또한 삼일포 기슭에는 4명의 신선이 놀고 간 

## [과제] Wikipedia documents에 대해 FAISS retrieval 실습해보기

In [33]:
# 3. 미리 올려둔 wiki docs 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

Cannot open cookies file '/tmp/cookies.txt': No such file or directory
--2022-07-28 14:25:42--  https://docs.google.com/uc?export=download&confirm=$(wget%20--quiet%20--save-cookies%20/tmp/cookies.txt%20--keep-session-cookies%20--no-check-certificate%20'https://docs.google.com/uc?export=download&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF'%20-O-%20%7C%20sed%20-rn%20's/.*confirm=([0-9A-Za-z_]+).*/%5C1%5Cn/p')&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF
Resolving docs.google.com (docs.google.com)... 172.217.161.238
Connecting to docs.google.com (docs.google.com)|172.217.161.238|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-07-28 14:25:42 ERROR 404: Not Found.



In [36]:
# First load wikipedia dump
import json

dump_path = "./wikipedia_documents.json"
with open(dump_path, 'r', encoding='UTF-8') as f:
    wiki = json.load(f)

In [37]:
# TODO: Generate a dense embedding of wiki documents from wikipedia_documents.json
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm.auto import tqdm, trange

eval_batch_size = 8
# Construt dataloader
corpus = [document['text'] for document_id, document in wiki.items()]
# Inference using the passage encoder to get dense embeddings

# wiki_embs = dense embedding of wiki documents
with torch.no_grad():
  wiki_embs = []
  for text in tqdm(corpus):
      p = tokenizer(text, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
      wiki_emb = p_encoder(**p).to('cpu').numpy() # use p_encoder
      wiki_embs.append(wiki_emb)

wiki_embs = torch.Tensor(wiki_embs).squeeze()  # (num_passage, emb_dim)

  0%|          | 0/60613 [00:00<?, ?it/s]

C:\Users\AI_15\AppData\Local\Temp\ipykernel_20016\1027532420.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  wiki_embs = torch.Tensor(wiki_embs).squeeze()  # (num_passage, emb_dim)


In [ ]:
# download wiki_p_embs 
import torch
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI" -O wiki_p_embs.pt && rm -rf /tmp/cookies.txt
wiki_embs = torch.load("wiki_p_embs.pt")

In [ ]:
# Answer: (60613, 768) (num_doc, hidden_dim)
wiki_embs.shape

torch.Size([60613, 768])

In [ ]:
# TODO: build faiss indexer using your own parameters
import faiss

num_clusters = ...
niter = ...
k = 5

# 1. Clustering
emb_dim = wiki_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(wiki_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(wiki_embs)
indexer.add(wiki_embs)

In [ ]:
# TODO: transform query to dense vector
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'

valid_q_seqs = ...

with torch.no_grad():
  q_encoder.eval()
  q_embs = ...
  
q_embs.shape  # (num_query, emb_dim)

In [ ]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

In [ ]:
print("[Search query]\n", q, "\n")
# 주의: 우리는 wiki 문서 전체에서 임의의 질문을 물어보는 것이므로 ground truth 가 존재하지 않습니다

d = D[0]
i = I[0]
for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
    print('\n')